# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")

# import NLP libraries
import re
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization

# import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # Define url pattern
    url_re = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Detect and replace urls
    detected_urls = re.findall(url_re, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # tokenize sentences
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # save cleaned tokens
    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]
    
    # remove stopwords
    STOPWORDS = list(set(stopwords.words('english')))
    clean_tokens = [token for token in clean_tokens if token not in STOPWORDS]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def build_pipeline():
    
    # build NLP pipeline - count words, tf-idf, multiple output classifier
    pipeline = Pipeline([
        ('vec', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 100, n_jobs = 6)))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline = build_pipeline()
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def build_report(pipeline, X_test, y_test):
    # predict on the X_test
    y_pred = pipeline.predict(X_test)
    
    # build classification report on every column
    performances = []
    for i in range(len(y_test.columns)):
        performances.append([f1_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
                             precision_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
                             recall_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro')])
    # build dataframe
    performances = pd.DataFrame(performances, columns=['f1 score', 'precision', 'recall'],
                                index = y_test.columns)   
    return performances

build_report(pipeline, X_test, y_test)

,f1 score,precision,recall
related,0.804852,0.804852,0.804852
request,0.900671,0.900671,0.900671
offer,0.994965,0.994965,0.994965
aid_related,0.781965,0.781965,0.781965
medical_help,0.919744,0.919744,0.919744
medical_products,0.951633,0.951633,0.951633
search_and_rescue,0.971926,0.971926,0.971926
security,0.982453,0.982453,0.982453
military,0.964144,0.964144,0.964144
child_alone,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
#parameters = {'clf__estimator__max_features':['sqrt', 0.5],
#              'clf__estimator__n_estimators':[50, 100]}
parameters = {'clf__estimator__n_estimators':[5]}
cv = GridSearchCV(estimator=pipeline, param_grid = parameters, cv = 2) #, n_jobs = 2)
cv.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
build_report(cv, X_test, y_test)

,f1 score,precision,recall
related,0.778914,0.778914,0.778914
request,0.877479,0.877479,0.877479
offer,0.994812,0.994812,0.994812
aid_related,0.726884,0.726884,0.726884
medical_help,0.916082,0.916082,0.916082
medical_products,0.950412,0.950412,0.950412
search_and_rescue,0.971163,0.971163,0.971163
security,0.981538,0.981538,0.981538
military,0.963839,0.963839,0.963839
child_alone,1.000000,1.000000,1.000000


In [10]:
cv.best_params_

{'clf__estimator__n_estimators': 5}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
pipeline_improved = Pipeline([
                                ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer()),
                                ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators = 100)))
                            ])
pipeline_improved.fit(X_train, y_train)
y_pred_improved = pipeline_improved.predict(X_test)
build_report(pipeline_improved, X_test, y_test)

,f1 score,precision,recall
related,0.767165,0.767165,0.767165
request,0.893653,0.893653,0.893653
offer,0.992829,0.992829,0.992829
aid_related,0.772353,0.772353,0.772353
medical_help,0.925389,0.925389,0.925389
medical_products,0.953616,0.953616,0.953616
search_and_rescue,0.973604,0.973604,0.973604
security,0.980317,0.980317,0.980317
military,0.969332,0.969332,0.969332
child_alone,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [ ]:
pickle.dump(pipeline, open('randomforest_model.pkl', 'wb'))

pickle.dump(pipeline_improved, open('adaboost_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.